In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [2]:
#function of parsing data from HTML

def parse():
    with open('imdb_most_popular_movies_dump.html') as file:   # reading data from the page
        data = file.read()
    soup = BeautifulSoup(data,'html.parser')   # creating a parser
    items = soup.findAll('td',class_='titleColumn')   # looking for a table that stores the information we need
    comps = []
    for item in items:
        comps.append([item.find('a').get_text(),item.find('span',class_='secondaryInfo').get_text()]) # geting the names of films and years of their release
    return comps

In [3]:
titles = parse() 
movie_numb = 0
# making a request to get the parameters of films to create a dictionary
response = requests.get(f'http://www.omdbapi.com/?apikey=c1f17bd4&t={titles[0][0].replace(" ", "+")}&y={titles[0][1][1:-1]}')
keys = dict(response.json()).keys()
film_dict = {key:{} for key in keys} # creating a dictionary of keys

for title in titles:
    try:
        # trying to get data about each film by title and year via API
        response = requests.get(f'http://www.omdbapi.com/?apikey=c1f17bd4&t={title[0].replace(" ", "+")}&y={title[1][1:-1]}')
        for key in keys:
            # writing parameters for the corresponding movie
            film_dict[key][f'{movie_numb}'] = response.json()[key]
    except:
        # if it doesn't work, try the same thing, but only by the name of the film (because the year of release in the IMBD and OMBD may not coincide)
        try:
            response = requests.get(f'http://www.omdbapi.com/?apikey=c1f17bd4&t={title[0].replace(" ", "+")}')
            for key in keys:
                film_dict[key][f'{movie_numb}'] = response.json()[key]
        # if the movie is not in the database, write down only its name and year
        except:
            film_dict["Title"][f'{movie_numb}'] = title[0]
            film_dict["Year"][f'{movie_numb}'] = title[1]
    # move on to the next movie
    movie_numb+=1


In [4]:
# making pandas DataFrame from the dictionary
df = pd.DataFrame(film_dict)
# sorting DataFrame by imdbRating
df['imdbRating'] = df['imdbRating'].apply(lambda x: None if x == 'N/A' else float(x))
df = df.sort_values(by=['imdbRating'], ascending=False)
# writing DataFrame to csv file
df.to_csv(r'films_top.csv', index = False)
# display the file on the screen
pd.read_csv('films_top.csv')

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,Drama,Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,...,80.0,9.3,"2,321,344",tt0111161,movie,NaN,"$28,699,976","Columbia Pictures, Castle Rock Entertainment",NaN,True
1,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay by), Francis Ford Coppo...","Marlon Brando, Al Pacino, James Caan, Richard ...",An organized crime dynasty's aging patriarch t...,...,100.0,9.2,"1,612,983",tt0068646,movie,NaN,"$134,966,411",Paramount Pictures,NaN,True
2,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama, Thriller",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker wreaks havo...,...,84.0,9.0,"2,294,527",tt0468569,movie,NaN,"$534,858,444",Syncopy,NaN,True
3,Pulp Fiction,1994,R,14 Oct 1994,154 min,"Crime, Drama",Quentin Tarantino,"Quentin Tarantino (stories), Roger Avary (stor...","Tim Roth, Amanda Plummer, Laura Lovelace, John...","The lives of two mob hitmen, a boxer, a gangst...",...,94.0,8.9,"1,819,518",tt0110912,movie,NaN,"$107,928,762","Miramax Films, A Band Apart, Jersey Films",NaN,True
4,Inception,2010,PG-13,16 Jul 2010,148 min,"Action, Adventure, Sci-Fi, Thriller",Christopher Nolan,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",A thief who steals corporate secrets through t...,...,74.0,8.8,"2,058,501",tt1375666,movie,NaN,"$292,576,195","Syncopy, Warner Bros.",NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Red Notice,2021,NaN,13 Nov 2020,NaN,"Action, Adventure, Comedy, Crime, Thriller",Rawson Marshall Thurber,Rawson Marshall Thurber,"Dwayne Johnson, Ryan Reynolds, Gal Gadot, Ritu...",An Interpol agent tracks the world's most want...,...,NaN,NaN,NaN,tt7991608,movie,NaN,NaN,NaN,NaN,True
96,The Suicide Squad,2021,NaN,06 Aug 2021,NaN,"Action, Adventure, Fantasy, Mystery, Sci-Fi",James Gunn,"James Gunn, John Ostrander (comic book)","Viola Davis, Margot Robbie, Taika Waititi, Ali...",Plot is unknown.,...,NaN,NaN,NaN,tt6334354,movie,NaN,NaN,NaN,NaN,True
97,Jumanji: Level One,2020,NaN,31 Dec 2020,NaN,Adventure,Lance Kawas,Dylan Sides,"Robert Laenen, Heather Fairbanks, Adam DeFilip...","Takes place in 1869, before the events of the ...",...,NaN,NaN,NaN,tt13249100,movie,NaN,NaN,NaN,NaN,True
98,Batman: Soul of the Dragon,2021,R,12 Jan 2021,83 min,"Animation, Action, Adventure, Crime",Sam Liu,Jeremy Adams (script),"Jamie Chung, Kelly Hu, Grey Griffin, David Giu...","In the 1970s, a missing teacher of martial art...",...,NaN,NaN,NaN,tt12885852,movie,NaN,NaN,NaN,NaN,True
